In [1]:
cd '/home/dpirvu/axion/'

/home/dpirvu/axion


In [ ]:
%matplotlib inline

import os,sys
sys.path.remove('/home/dpirvu/DarkPhotonxunWISE/hmvec-master')
sys.path.append('/home/dpirvu/axion/hmvec-master/')
print([ii for ii in sys.path])

import hmvec as hm
from compute_power_spectra import *
from params import *
from plotting import *

['/home/dpirvu/axion', '/cm/shared/apps/python/python37/lib/python37.zip', '/cm/shared/apps/python/python37/lib/python3.7', '/cm/shared/apps/python/python37/lib/python3.7/lib-dynload', '', '/home/dpirvu/.local/lib/python3.7/site-packages', '/cm/shared/apps/python/python37/lib/python3.7/site-packages', '/cm/shared/apps/python/python37/lib/python3.7/site-packages/IPython/extensions', '/home/dpirvu/.ipython', '/home/dpirvu/axion/hmvec-master/']


In [ ]:
ellMax = 9600
ells = np.arange(ellMax)

ellFinal = 6001

nZs = 50
nMs = 100
zMin = 0.005
zreio = 1.9
zMax = zreio

ms  = np.geomspace(1e11,1e17, nMs) # masses
zs  = np.linspace(0.005,zreio,nZs) # redshifts
ks  = np.geomspace(1e-4,1e3, 5001) # wavenumbers

rscale = False

baseline = ghztoev(353)
units = xov(baseline) * baseline


In [ ]:
# Select electron profile
for name in ['battagliaAGN', 'NFW']:
    for color in ['blue', 'green']:
        unwise_color = color
        for choose in [21]:
            choose_dict = choose
            print('Doing ', name, unwise_color, choose_dict)

            if unwise_color=='blue': pathdndz = "/home/dpirvu/DarkPhotonxunWISE/dataHOD/normalised_dndz_cosmos_0.txt"
            elif unwise_color=='green': pathdndz = "/home/dpirvu/DarkPhotonxunWISE/dataHOD/normalised_dndz_cosmos_1.txt"

            # Halo Model
            hcos  = hm.HaloModel(zs, ks, ms=ms, mass_function='tinker', mdef='vir', concmode='BHATTACHARYA', unwise_color=unwise_color, choose_dict=choose_dict)
            chis  = hcos.comoving_radial_distance(zs)
            Hz    = hcos.h_of_z(zs)
            nzm   = hcos.get_nzm()
            biases= hcos.get_bh()
            dvols = get_volume_conv(chis, Hz)

            hcos.add_hod(name=unwise_color)
            Ncs  = hcos.hods[unwise_color]['Nc']
            Nss  = hcos.hods[unwise_color]['Ns']
            ngal = hcos.hods[unwise_color]['ngal']
            hod, uc, us = hcos._get_hod_common(unwise_color)

            dndz, zs, N_gtot, W_g, zsHOD, dndzHOD = get_dndz(zs, pathdndz, dvols)

            uk_g   = (      Ncs[None,:,None] + us     * Nss[None,:,None]    ) / ngal[:,None,None]
            uk_gsq = (2.*us*Nss[None,:,None] + us**2. * Nss[None,:,None]**2.) / ngal[:,None,None]**2.
            PzkLin = hcos._get_matter_power(zs, ks, nonlinear=False)

            Pzell, uell_g, uell_gsq = get_fourier_to_multipole_functs(zs, ms, ks, chis, W_g, ellMax, uk_g, uk_gsq, PzkLin)
            print('Done turning into multipoles.')

            data_save = np.zeros((len(dictKey), ellFinal))
            for mind, MA in enumerate(dictKey):
                zMin, zMax, rMin, rMax = chooseModel(MA, modelParams)
                zMax = min(zreio, zMax)

                # HERE YOU NEED TO IMPORT THE PROBABILITY FILE
                path_params0 = np.asarray([MA, nZs, zMin, zMax, ellMax, rscale])
                rcross, prob, utheta, avtau, dtaudz, uell0 = np.load(data_path(*path_params0), allow_pickle=True)

                screeningProbell = (prob * utheta)[None,...] * uell0
                zell_tau = (screeningProbell[:ellMax]).transpose(1,2,0) * np.sqrt((4.*np.pi)/(2*ells+1))[None,None,:]

                # Assemble power spectra
                int_uell_g_1h = np.trapz(nzm[...,None] * uell_gsq         , ms, axis=1)
                int_zell_taug = np.trapz(nzm[...,None] * zell_tau * uell_g, ms, axis=1)

                int_uell_g_2h = np.trapz(nzm[...,None] * biases[...,None] * uell_g  , ms, axis=1)
                int_zell_tau  = np.trapz(nzm[...,None] * biases[...,None] * zell_tau, ms, axis=1)

                Cell_taug_1h  = np.trapz(dvols[:,None] * int_zell_taug                       , zs, axis=0)
                Cell_taug_2h  = np.trapz(dvols[:,None] * int_zell_tau * int_uell_g_2h * Pzell, zs, axis=0)
                Cell_taug_tot = Cell_taug_1h + Cell_taug_2h

                data_save[mind] = Cell_taug_tot[:ellFinal] * TCMB * units
                print('Done', mind, 'out of', len(dictKey))

#            np.save('/home/dpirvu/axion/data_axion_Tg_'+name+'_'+unwise_color+'_HOD'+str(choose_dict)+'.npy', data_save)
            print('Done ', name, unwise_color, choose_dict)


In [ ]:
bluetest  = np.load('/home/dpirvu/axion/data_axion_Tg_battagliaAGN_blue_HOD21.npy',  allow_pickle=True)
greentest = np.load('/home/dpirvu/axion/data_axion_Tg_battagliaAGN_green_HOD21.npy', allow_pickle=True)
